In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
# rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
fig_path = "/home/chuyu/Notebooks/project_place_cell/figures/output/sfigure8/rotation_rectangular"
mkpath(fig_path)

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")

In [ ]:
experiment_filename_1 = "20230226_122159"
server_1 = 9

experiment_filename_2 = "20230226_141707"
server_2 = 9

experimenter = "lorenz"
analyzer = "chuyu"

In [ ]:
ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

In [ ]:
NMF_filename = joinpath(data_path(ds_save_cy_1), "NMF_merge.h5")
NMF_file = h5open(NMF_filename, "r")
global Z_all = HDF5.readmmap(NMF_file["Z_all"])
global X_all = HDF5.readmmap(NMF_file["X_all"])
global Y_all = HDF5.readmmap(NMF_file["Y_all"])
global neuron_label = HDF5.readmmap(NMF_file["neuron_label"])
global A_dF_1 = HDF5.readmmap(NMF_file["A_dF"]); # bs update
global A_dFF_1 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
global A_baseline_1 = HDF5.readmmap(NMF_file["A_baseline"]); # bs update
close(NMF_file)

# for multi sessions
NMF_filename = joinpath(data_path(ds_save_cy_2), "NMF_merge.h5") # bs update
NMF_file = h5open(NMF_filename, "r") # bs update
global A_dF_2 = HDF5.readmmap(NMF_file["A_dF"]); # bs update
global A_dFF_2 = HDF5.readmmap(NMF_file["A_dFF"]); # bs update
global A_baseline_2 = HDF5.readmmap(NMF_file["A_baseline"]); # bs update
close(NMF_file) # bs update

n_neuron = length(X_all);

# whether individual roi belongs to a certain region
region_bool_filename = joinpath(data_path(ds_save_cy_1), "region_roi_bool.h5")
region_bool_file = h5open(region_bool_filename, "r")
global region_names = read(region_bool_file, "region_names")
global region_roi_bool = read(region_bool_file, "region_roi_bool")
close(region_bool_file)


# for one merged cell, it belongs to telecephalon if at least one of its roi belongs to telencephalon
region_roi_bool_tel = region_roi_bool[:,findall(region_names .== "Telencephalon -")][:,1]
whether_tel = falses(n_neuron)
for which_neuron in Int32.(numpy.unique(neuron_label)[1:end-1])
    if sum(region_roi_bool_tel[neuron_label.==which_neuron]) >0
        whether_tel[which_neuron] = true
    end
end


all_files = readdir(data_path(ds_save_cy_1))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
specificity_1 = HDF5.readmmap(file["specificity"])
valid_roi_1 = HDF5.readmmap(file["valid_neurons"])
entropy_1 = HDF5.readmmap(file["entropy_all"])
place_cell_index_1 = HDF5.readmmap(file["place_cell_index"])
bool_index_1 = HDF5.readmmap(file["bool_index"])

close(file)
    
    
all_files = readdir(data_path(ds_save_cy_2))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
file = h5open(info_filename, "r")
specificity_2 = HDF5.readmmap(file["specificity"])
valid_roi_2 = HDF5.readmmap(file["valid_neurons"])
entropy_2 = HDF5.readmmap(file["entropy_all"])
place_cell_index_2 = HDF5.readmmap(file["place_cell_index"])
bool_index_2 = HDF5.readmmap(file["bool_index"])
close(file)
n_bins = size(place_map_all_1, 1)
    
    info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
    file = h5open(info_filename, "r")
    global place_map_all_2 = HDF5.readmmap(file["place_map_all"])
    close(file)
        

    


img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;
    
global img_bg_end_1 = img_bg_1[:,:,end]
global img_bg_end_2 = img_bg_2[:,:,end]


global w = size(img_bg_end_1, 1)
global l = size(img_bg_end_1, 2)
    
    

# whether individual roi belongs to a certain region
save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
tel_place_cell_index  = HDF5.readmmap(file["tel_place_cell_index"])
peak_loc_map_1 = HDF5.readmmap(file["peak_loc_map_1"])
peak_loc_map_2 = HDF5.readmmap(file["peak_loc_map_2"])
precise_loc = false
if haskey(file, "peak_loc_map_1_precise")
    precise_loc = true
    peak_loc_map_1_precise_array = HDF5.readmmap(file["peak_loc_map_1_precise"])
    peak_loc_map_2_precise_array = HDF5.readmmap(file["peak_loc_map_2_precise"])
end
close(file)



plot_loc(confined_place_cell_index_1; valid_neurons = valid_roi_1, label= length(confined_place_cell_index_1))




In [ ]:
for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_bins).h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
x_fish_sweep_mean_1 = read(for_place_calculation_file,"x_fish_sweep_mean")
y_fish_sweep_mean_1 = read(for_place_calculation_file,"y_fish_sweep_mean")
close(for_place_calculation_file)

for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_2), "for_place_calculation_chamber_geometry_$(experiment_filename_2)_n$(n_bins).h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
x_fish_sweep_mean_2 = read(for_place_calculation_file,"x_fish_sweep_mean")
y_fish_sweep_mean_2 = read(for_place_calculation_file,"y_fish_sweep_mean")
close(for_place_calculation_file)

In [ ]:
# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
chamber_roi_1 = read(orientation_correction_file,"chamber_roi")
countour_array_1 = read(orientation_correction_file,"countour_array")
center_loc_1 = read(orientation_correction_file,"center_loc")
close(orientation_correction_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
chamber_roi_2 = read(orientation_correction_file,"chamber_roi")
countour_array_2 = read(orientation_correction_file,"countour_array")
center_loc_2 = read(orientation_correction_file,"center_loc")

close(orientation_correction_file)

In [ ]:
bin_interval = x_bins[2] - x_bins[1]

KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_1'))
class_1 = KMeans_features.labels_;
KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_2'))
class_2 = KMeans_features.labels_;

fig = figure()
scatter((peak_loc_map_1[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)].-0.5).*bin_interval, (peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)].-0.5).*bin_interval, c=class_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_1)], cmap="Dark2", s=10, alpha=0.3)
# colorbar()
imshow(img_bg_end_1', cmap="binary", vmax=2000)
axis("off")
title(experiment_filename_1)

fig = figure()
scatter((peak_loc_map_2[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)].-0.5).*bin_interval, (peak_loc_map_2[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)].-0.5).*bin_interval, c=class_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_2)], cmap="Dark2", s=10, alpha=0.3)
# colorbar()
imshow(img_bg_end_2', cmap="binary", vmax=2000)
axis("off")


In [ ]:
if ! precise_loc
    @pyimport skimage.transform as skimage_transform
    function resize_map(which_map, bin_interval)
        map_resized = skimage_transform.rescale(which_map, bin_interval, anti_aliasing=false,clip =true,preserve_range=true)
        return map_resized
    end


    peak_loc_map_1_precise = []
    peak_loc_map_2_precise = []
    @showprogress for which_neuron in confined_place_cell_index_1
        which_map = place_map_all_1[:,:,which_neuron]
        map_resized = resize_map(which_map, bin_interval)
        components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = 8/10, components_size_threshold = 20)
        img_label_resized = resize_map(img_label_valid, bin_interval)
        img_label_resized = Int32.(round.(img_label_resized))

        peak_values = [numpy.nanpercentile(which_map[img_label_valid .== valid_components[i]], 95) for i in 1:length(valid_components)]
        peak_component = valid_components[findall(peak_values .== maximum(peak_values))]

        map_resized[img_label_resized .!=peak_component].=NaN;
        append!(peak_loc_map_1_precise, [find_com(map_resized)])

    end
    @showprogress for which_neuron in confined_place_cell_index_2
        which_map = place_map_all_2[:,:,which_neuron]
        map_resized = resize_map(which_map, bin_interval)
        components_peaks, img_label_valid, valid_components = map_components_peak(which_map; threshold = 8/10, components_size_threshold = 20)
        img_label_resized = resize_map(img_label_valid, bin_interval)
        img_label_resized = Int32.(round.(img_label_resized))

        peak_values = [numpy.nanpercentile(which_map[img_label_valid .== valid_components[i]], 95) for i in 1:length(valid_components)]
        peak_component = valid_components[findall(peak_values .== maximum(peak_values))]

        map_resized[img_label_resized .!=peak_component].=NaN;
        append!(peak_loc_map_2_precise, [find_com(map_resized)])
    end

    peak_loc_map_1_precise_array = numpy.array(peak_loc_map_1_precise)'
    peak_loc_map_2_precise_array = numpy.array(peak_loc_map_2_precise)';
    
    
    h5open(joinpath(data_path(ds_save_analyzer_1), "compare_map_results_original.h5"), "r+") do file
        if haskey(file, "peak_loc_map_1_precise")
            delete_object(file, "peak_loc_map_1_precise")
        end
        if haskey(file, "peak_loc_map_2_precise")
            delete_object(file, "peak_loc_map_2_precise")
        end
        file["peak_loc_map_1_precise"] = collect(peak_loc_map_1_precise_array)
        file["peak_loc_map_2_precise"] = collect(peak_loc_map_2_precise_array)
    end;
end

In [ ]:
@pyimport matplotlib.cm as cm
@pyimport sklearn.metrics as metrics
@pyimport colorsys
@pyimport cmocean

# hsv_cmap = cm.get_cmap("hsv", 360)
hsv_cmap = cmocean.cm.phase

In [ ]:
function hsv_control(rgb_color, s, v)
    color_hsv = colorsys.rgb_to_hsv(rgb_color[1],rgb_color[2],rgb_color[3])
    color_rgb = colorsys.hsv_to_rgb(color_hsv[1],s,v)
    return color_rgb
end

In [ ]:
distance_center_1 = distance_from(collect(peak_loc_map_1_precise_array'), center_loc_1);
distance_edge_1, _ = distance_to_feature(collect(peak_loc_map_1_precise_array'), countour_array_1);

angle_center_1 = angle_from(peak_loc_map_1_precise_array', center_loc_1);
angle_center_1[angle_center_1.<0]=angle_center_1[angle_center_1.<0].+2*pi;

In [ ]:
rgb_color = hsv_cmap(Int32.(round.(180*angle_center_1/pi)))
hsv_s = [(maximum(distance_edge_1) .-distance_edge_1[i])/maximum(distance_edge_1)+1 for i in 1:size(rgb_color, 1)];
hsv_v = [(maximum(distance_edge_1) .-distance_edge_1[i])/maximum(distance_edge_1)+0.3 for i in 1:size(rgb_color, 1)];
hsv_s[hsv_s.!=1].=1
hsv_v[hsv_v.>1].=1
rgb_color_s = [hsv_control(rgb_color[i,:], hsv_s[i],hsv_v[i]) for i in 1:size(rgb_color, 1)];

In [ ]:
# peak locations

chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

countour_array = countour_array_1

fig = figure()
scatter(peak_loc_map_1_precise_array[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)], peak_loc_map_1_precise_array[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)], c=rgb_color_s[whether_in(confined_place_cell_index_1, confined_place_cell_index_1)], s=60, alpha=0.5,linewidth = 0)

axis("off")
axis("equal")
plot(vcat(countour_array[1:50:end,1][end], countour_array[1:50:end,1]),vcat(countour_array[1:50:end,2][end], countour_array[1:50:end,2]),c="k", alpha=0.1)
    xlim(xmin,xmax)
    ylim(ymax,ymin)
# fig.savefig(joinpath(fig_path, "com.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "com_1.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);




chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

countour_array = countour_array_2

fig = figure()
scatter(peak_loc_map_2_precise_array[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)], peak_loc_map_2_precise_array[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)], c=rgb_color_s[whether_in(confined_place_cell_index_1, confined_place_cell_index_2)], s=60, alpha=0.5,linewidth = 0)
imshow(chamber_roi_mask', origin="lower",alpha=0)

axis("off")
axis("equal")

    xlim(xmin,xmax)
    ylim(ymax,ymin)
plot(vcat(countour_array[1:50:end,1][end], countour_array[1:50:end,1]),vcat(countour_array[1:50:end,2][end], countour_array[1:50:end,2]),c="k", alpha=0.1)
# fig.savefig(joinpath(fig_path, "com.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "com_2.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

In [ ]:
fig_path

In [ ]:
distance_center_2 = distance_from(collect(peak_loc_map_2_precise_array'), center_loc_2);
distance_edge_2, _ = distance_to_feature(collect(peak_loc_map_2_precise_array'), countour_array_2);

angle_center_2 = angle_from(peak_loc_map_2_precise_array', center_loc_2);
angle_center_2[angle_center_2.<0]=angle_center_2[angle_center_2.<0].+2*pi;

In [ ]:
rgb_color = hsv_cmap(Int32.(round.(180*angle_center_2/pi)))
hsv_s = [(maximum(distance_edge_2) .-distance_edge_2[i])/maximum(distance_edge_2) for i in 1:size(rgb_color, 1)];
hsv_v = [(maximum(distance_edge_2) .-distance_edge_2[i])/maximum(distance_edge_2)+0.3 for i in 1:size(rgb_color, 1)];
hsv_s[hsv_s.!=1].=1
hsv_v[hsv_v.>1].=1
rgb_color_s = [hsv_control(rgb_color[i,:], hsv_s[i],hsv_v[i]) for i in 1:size(rgb_color, 1)];

In [ ]:
# peak locations

chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

countour_array = countour_array_1

fig = figure()
scatter(peak_loc_map_1_precise_array[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)], peak_loc_map_1_precise_array[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)], c=rgb_color_s[whether_in(confined_place_cell_index_2, confined_place_cell_index_1)], s=60, alpha=0.5,linewidth = 0)

axis("off")
axis("equal")
plot(vcat(countour_array[1:50:end,1][end], countour_array[1:50:end,1]),vcat(countour_array[1:50:end,2][end], countour_array[1:50:end,2]),c="k", alpha=0.1)
    xlim(xmin,xmax)
    ylim(ymax,ymin)
# fig.savefig(joinpath(fig_path, "com.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "com_1_2.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);




chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

countour_array = countour_array_2

fig = figure()
scatter(peak_loc_map_2_precise_array[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)], peak_loc_map_2_precise_array[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)], c=rgb_color_s[whether_in(confined_place_cell_index_2, confined_place_cell_index_2)], s=60, alpha=0.5,linewidth = 0)
imshow(chamber_roi_mask', origin="lower",alpha=0)

axis("off")
axis("equal")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
plot(vcat(countour_array[1:50:end,1][end], countour_array[1:50:end,1]),vcat(countour_array[1:50:end,2][end], countour_array[1:50:end,2]),c="k", alpha=0.1)
# fig.savefig(joinpath(fig_path, "com.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
fig.savefig(joinpath(fig_path, "com_2_2.pdf"), bbox_inches="tight",transparent = true,pad_inches = 0);

# Save source data

In [ ]:
@pyimport pandas as pd

In [ ]:
function fill_length(vec, desired_length)
    [vec; fill(NaN, desired_length-length(vec))]
end

In [ ]:
rgb_color = hsv_cmap(Int32.(round.(180*angle_center_1/pi)))
hsv_s = [(maximum(distance_edge_1) .-distance_edge_1[i])/maximum(distance_edge_1)+1 for i in 1:size(rgb_color, 1)];
hsv_v = [(maximum(distance_edge_1) .-distance_edge_1[i])/maximum(distance_edge_1)+0.3 for i in 1:size(rgb_color, 1)];
hsv_s[hsv_s.!=1].=1
hsv_v[hsv_v.>1].=1
rgb_color_s_1 = [hsv_control(rgb_color[i,:], hsv_s[i],hsv_v[i]) for i in 1:size(rgb_color, 1)];

In [ ]:
rgb_color = hsv_cmap(Int32.(round.(180*angle_center_2/pi)))
hsv_s = [(maximum(distance_edge_2) .-distance_edge_2[i])/maximum(distance_edge_2) for i in 1:size(rgb_color, 1)];
hsv_v = [(maximum(distance_edge_2) .-distance_edge_2[i])/maximum(distance_edge_2)+0.3 for i in 1:size(rgb_color, 1)];
hsv_s[hsv_s.!=1].=1
hsv_v[hsv_v.>1].=1
rgb_color_s_2 = [hsv_control(rgb_color[i,:], hsv_s[i],hsv_v[i]) for i in 1:size(rgb_color, 1)];

In [ ]:
data_dict = Dict(
    "S1 PC in S1, COM x" => peak_loc_map_1_precise_array[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)],
    "S1 PC in S1, COM y" => peak_loc_map_1_precise_array[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)],
    "S1 PC in S2, COM x" => peak_loc_map_2_precise_array[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)],
    "S1 PC in S2, COM y" => peak_loc_map_2_precise_array[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_1)],
    "S1 PC in S1, color" => rgb_color_s_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_1)],
    "S1 PC in S2, color" => rgb_color_s_1[whether_in(confined_place_cell_index_1, confined_place_cell_index_2)],


    "S2 PC in S1, COM x" => peak_loc_map_1_precise_array[1,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)],
    "S2 PC in S1, COM y" => peak_loc_map_1_precise_array[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_2)],
    "S2 PC in S2, COM x" => peak_loc_map_2_precise_array[1,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)],
    "S2 PC in S2, COM y" => peak_loc_map_2_precise_array[2,whether_in(confined_place_cell_index_2, confined_place_cell_index_2)],
    "S2 PC in S1, color" => rgb_color_s_2[whether_in(confined_place_cell_index_2, confined_place_cell_index_1)],
    "S2 PC in S2, color" => rgb_color_s_2[whether_in(confined_place_cell_index_2, confined_place_cell_index_2)]
)
max_length = maximum([length(data_dict[key]) for key in keys(data_dict)])
for key in keys(data_dict)
    data_dict[key] = fill_length(data_dict[key], max_length)
end

df = pd.DataFrame.from_dict(data_dict)
df = df.sort_index(axis=1)

In [ ]:
@pyimport os
@pyimport openpyxl

save_path = "../../../figures/source_data/Source Data Fig. 5.xlsx"
sheet_name = "Fig. 5n"

exists = os.path.isfile(save_path)
if exists
    writer = pd.ExcelWriter(save_path, engine = "openpyxl", mode="a", if_sheet_exists="replace")
    df.to_excel(writer, sheet_name = sheet_name)
    writer.close()
else
    df.to_excel(save_path, sheet_name)  
end

# Save example maps

In [ ]:
chosen_neruon_index = [31501, 36879, 8442, 4109, 41762, 50191, 13380, 24783, 3601]

In [ ]:
mask_valid = isfinite.(place_map_all_1[:,:,1])

mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_1))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for real_cell_index in chosen_neruon_index
    fig = figure()
    map = place_map_all_1[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,real_cell_index]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower")
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map1.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end

In [ ]:
mask_valid = isfinite.(place_map_all_2[:,:,1])

mask_valid_index = findall(mask_valid)
mask_valid_x = [x[1] for x in mask_valid_index]
mask_valid_y = [x[2] for x in mask_valid_index];
mask_valid_x_min = minimum(mask_valid_x)
mask_valid_x_max = maximum(mask_valid_x)
mask_valid_y_min = minimum(mask_valid_y)
mask_valid_y_max = maximum(mask_valid_y);

In [ ]:
chamber_roi_mask = Float32.(copy(chamber_roi_2))
chamber_roi_mask[chamber_roi_mask.!=0].= NaN32
imshow(chamber_roi_mask)

In [ ]:
# example cells

valid_frame = isnan.(chamber_roi_mask[:,:,1]); # bs update
ymin = findfirst(sum(valid_frame;dims=1).!=0)[2]-10; # bs update
ymax = findlast(sum(valid_frame;dims=1).!=0)[2]+10; # bs update
xmin = findfirst(sum(valid_frame;dims=2).!=0)[1]-10; # bs update
xmax = findlast(sum(valid_frame;dims=2).!=0)[1]+10; # bs update

for real_cell_index in chosen_neruon_index
    fig = figure()
    map = place_map_all_2[mask_valid_x_min:mask_valid_x_max,mask_valid_y_min:mask_valid_y_max,real_cell_index]
    sp1 = imshow(map', extent = [x_bins[mask_valid_x_min], x_bins[mask_valid_x_max+1], y_bins[mask_valid_y_min], y_bins[mask_valid_y_max+1]], cmap="jet", origin="lower")
    # imshow(chamber_roi_mask', cmap="binary", origin="lower", vmin=0, vmax=3)
    axis("off")
    xlim(xmin,xmax)
    ylim(ymax,ymin)
    tight_layout()
    fig.savefig(joinpath(fig_path, "$(experiment_filename_1)_$real_cell_index map2.png"), bbox_inches = "tight",transparent = true,pad_inches = 0)
end